In [ ]:
import os

from trulens.connectors.snowflake import SnowflakeConnector

os.environ["TRULENS_OTEL_TRACING"] = "1"
connection_params = {  # creds for QA6
    "account": "...",
    "user": "...",
    "password": "...",
    "database": "...",
    "warehouse": "...",
    "role": "...",
}

sf_connector = SnowflakeConnector(**connection_params)

In [ ]:
from trulens.apps.app import TruApp
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class TestApp:
    @instrument(span_type=SpanAttributes.SpanType.MAIN)
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument(attributes={"nested_attr1": "value1"})
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested2_ret": ret,
            "nested2_args[1]": args[1],
        }
    )
    def nested2(self, query: str) -> str:
        nested_result = ""

        try:
            nested_result = self.nested3(query)
        except Exception:
            pass

        return f"nested2: {nested_result}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested3_ex": exception.args if exception else None,
            "nested3_ret": ret,
            "selector_name": "special",
            "cows": "moo",
        }
    )
    def nested3(self, query: str) -> str:
        if query == "throw":
            raise ValueError("nested3 exception")
        return "nested3"


app = TestApp()

In [ ]:
TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
    object_type="RANDOM_UNSUPPORTED",
)

TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
)

TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
    main_method=app.respond_to_query,
)
# these should all throw exceptions

In [ ]:
tru_recorder = TruApp(
    app,
    app_name="custom_app",
    app_version="v1",
    connector=sf_connector,
    main_method=app.respond_to_query,
    # object_type default to "EXTERNAL AGENT" when snowflake connector is used
)

print(tru_recorder.snowflake_object_type)  # EXTERNAL AGENT
print(tru_recorder.snowflake_object_name)  # CUSTOM_APP

version_df = tru_recorder.snowflake_app_dao.list_agent_versions("custom_app")
version_df

In [ ]:
tru_recorder.delete_snowflake_app()

assert tru_recorder.snowflake_app_dao.list_agents().empty

In [ ]:
tru_recorder_v1 = TruApp(
    app,
    app_name="custom_app_multi_ver",
    app_version="v1",
    connector=sf_connector,
    main_method=app.respond_to_query,
)

assert tru_recorder_v1.snowflake_app_dao.check_agent_exists(
    "custom_app_multi_ver"
)

# Create app version 2.
tru_recorder_v2 = TruApp(
    app,
    app_name="custom_app_multi_ver",
    app_version="v2",
    connector=sf_connector,
    main_method=app.respond_to_query,
)

assert tru_recorder_v2.snowflake_app_dao is not None

assert tru_recorder_v2.snowflake_app_dao.check_agent_exists(
    "custom_app_multi_ver"
)

versions_df_1 = tru_recorder_v1.snowflake_app_dao.list_agent_versions(
    "custom_app_multi_ver"
)

versions_df_2 = tru_recorder_v2.snowflake_app_dao.list_agent_versions(
    "custom_app_multi_ver"
)

## Add runs to agent

In [ ]:
from trulens.core.run import Run

run_config = Run.RunConfig(
    description="desc",
    dataset_fqn="db.schema.table",
    dataset_col_spec=None,
)  # type: ignore
new_run = tru_recorder.add_run(run_name="test_run_1", run_config=run_config)

run = tru_recorder.get_run("test_run_1")

assert run.run_name == "test_run_1"
assert run.run_name == new_run.run_name
# so forth

print(run.model_dump())
print(new_run.model_dump())

In [ ]:
run_config_2 = Run.RunConfig(
    description="desc_2",
    dataset_fqn="db.schema.table",
    dataset_col_spec=None,
)
run_2 = tru_recorder.add_run(run_name="test_run_2", run_config=run_config_2)

runs = tru_recorder.list_runs()

In [ ]:
run_2.delete()
runs = tru_recorder.list_runs()